In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
data = pd.read_csv("Data/Data.csv")

# PRE-PROCESSING

In [3]:
# Loại bỏ các ký tự đặc biệt
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)
    return text

data['comment_text_cleaned'] = data['comment_text'].apply(clean_text)

In [4]:
teencode = {
    "không": ["hong", "hok", "khom", "khum", "k", "ko", "khong", "hem", "no", "hơm", "hôn","hk", "hông"],
    "biết": ["bít", "biếc", "bt"],
    "rồi": ["ròi", "gòi", "rùi", "rui", "roi", "goy", "goi", "gui", "gùi", "ời", "oy", "òy", "roài", "ùi","r"],
    "tôi": ["teo", "toy", "tui", "t"],
    "em": ["iem", "e"],
    "yêu": ["iu", "ieo"],
    "ôi" : ["ưi", "ui"],
    "ừm" :["um", "ờm"],
    "ừ" : ["ò", "uk", "ầu", "ờ"],
    "chồng" : ["ck", "chòng"],
    "vợ": ["vk", "zợ", "zk"],
    "ông" : ["ong", "ô"],
    "anh": ["a"],
    "bé" :["pé"],
    "chị" : ["chụy", "cj", "cỵ", "cị"],
    "giờ" :["h"],
    "ch":["ck"],
    "ph":["f"],
    "th":["tk"],
    "nh":["nk"],
    "gì": ["j", "giề", "rì", "zì"],
    "muốn" : ["mún"],
    "học" : ["hc"],
    "bạn" :["bẹng", "bạng", "bn", "pạn"],
    "mày" :["m", "mèy", "mài"],
    "đang" : ["aq", "đaq", "đag", "dang", "dag"],
    "đường": ["đg"],
    "làm":['nàm', "lèm"],
    "cao":['kao'],
    'ch':['tr'],
    'tr':['ch'],
    'vậy':['zạy', 'zay', 'vại', 'dọ', 'dạ', 'zị', 'zậy', 'zọ', "v", 'z', 'dị'],
    "nội":['nụi'],
    "quen" :['wen'],
    "hoài" : ["quài"],
    "ỉa":['ẻ'],
    "hả":['hỏ', 'hẻ', 'hở'],
    "bà" :['pà'],
    "khùng" :["khùm", "khùn"],
    "điên" :["đin", "điêng"],
    "ngu" :["nguk"],
    "ngốc" :["ngok"],
    "được" :["dc", "dk", "duoc", "đc"],
    "trời" : ["chòi", "chời", "trùi", "tr"],
    "cũng" : ["cũm", "cx"],
    "nha": ["nhoa", "nho", "nhe"],
    "diễn viên" : ["dv"],
    "ca sĩ" : ["ca sỉ", "cs"],
    "mọi người" : ["mọi ngừi", "mn"],
    "thằng" : ["tk","thg"],
    "người": ["ngừi", "ng"],
    "quá": ["qá", "wa", "quớ"]
}


reverse_teencode = {teencode_word: full_word for full_word, teencode_list in teencode.items() for teencode_word in teencode_list}

def replace_teencode(text):
    words = text.split()
    replaced_words = [reverse_teencode.get(word, word) for word in words]
    return ' '.join(replaced_words)

data['comment_text_cleaned'] = data['comment_text_cleaned'].apply(replace_teencode)


In [5]:
data['comment_text_cleaned'] = data['comment_text_cleaned'].str.lower()

In [6]:
# Loại bỏ các từ dừng (stop words)

with open('Vietnamese_data/vietnamese-stop-words.txt', 'r', encoding='utf-8') as file:
    stop_words = set(file.read())

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

data['comment_text_cleaned'] = data['comment_text_cleaned'].apply(remove_stopwords)

In [7]:
data['comment_text_cleaned'].iloc[2100] 


'phân tích cứ như tích 1 đống phân trc đây thì giọng sáng hơn âm vực được tầm 3 quãng cơ mà hát thì đầy lưỡi phát âm bị bẹt không tròn còn giờ thì không sáng yếu được cái chữa được cái phát âm lỗi bảo là dở thì không phải nhưng kiểu chất giọng lạ lạ 1 chút kiểu lam trường chứ chả có vẹo gì mà phân tích về giọng của tún'

# MODEL

In [8]:
from sklearn.model_selection import train_test_split


X_train, X_temp, y_train, y_temp  = train_test_split(data['comment_text_cleaned'],data['label'], test_size=0.4, random_state=42)

X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [9]:
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
X_val_vec = vectorizer.transform(X_val)

### DECISION TREE

In [11]:
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_vec, y_train)
y_test_pred = model.predict(X_test_vec)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred)

print(f"Accuracy test: {test_accuracy:.2f}")
print(f"Class report test:")
print(test_class_report)

y_val_pred = model.predict(X_val_vec)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_class_report = classification_report(y_val, y_val_pred)

print(f"Accuracy val: {val_accuracy:.2f}")
print(f"Class report validation:")
print(val_class_report)

Accuracy test: 0.60
Class report test:
              precision    recall  f1-score   support

          -1       0.52      0.45      0.48       144
           0       0.66      0.68      0.67       296
           1       0.50      0.55      0.52       102

    accuracy                           0.60       542
   macro avg       0.56      0.56      0.56       542
weighted avg       0.59      0.60      0.59       542

Accuracy val: 0.60
Class report validation:
              precision    recall  f1-score   support

          -1       0.46      0.43      0.44       134
           0       0.65      0.70      0.68       299
           1       0.62      0.52      0.57       109

    accuracy                           0.60       542
   macro avg       0.58      0.55      0.56       542
weighted avg       0.60      0.60      0.60       542



In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score,confusion_matrix, classification_report

In [13]:
param_grid = {
    'criterion': ['gini', 'entropy'], 
    'max_depth': [None, 10, 20, 30, 40, 50], 
    'min_samples_split': [2, 5, 10], 
    'min_samples_leaf': [1, 2, 4]  
}

# Sử dụng GridSearchCV để tìm kiếm tham số tốt nhất
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_vec, y_train)

print("Tham số tốt nhất:", grid_search.best_params_)

best_model = grid_search.best_estimator_
y_test_pred = best_model.predict(X_test_vec)
test_accuracy = accuracy_score(y_test, y_test_pred)
test_class_report = classification_report(y_test, y_test_pred)

print(f"Accuracy test: {test_accuracy:.2f}")
print(f"Class report test:")
print(test_class_report)

y_val_pred = best_model.predict(X_val_vec)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_class_report = classification_report(y_val, y_val_pred)

print(f"Accuracy val: {val_accuracy:.2f}")
print(f"Class report validation:")
print(val_class_report)

Tham số tốt nhất: {'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10}
Accuracy test: 0.61
Class report test:
              precision    recall  f1-score   support

          -1       0.69      0.20      0.31       144
           0       0.60      0.89      0.72       296
           1       0.55      0.35      0.43       102

    accuracy                           0.61       542
   macro avg       0.62      0.48      0.49       542
weighted avg       0.62      0.61      0.56       542

Accuracy val: 0.59
Class report validation:
              precision    recall  f1-score   support

          -1       0.51      0.19      0.27       134
           0       0.60      0.87      0.71       299
           1       0.62      0.33      0.43       109

    accuracy                           0.59       542
   macro avg       0.58      0.46      0.47       542
weighted avg       0.58      0.59      0.55       542



In [14]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_vec, y_train)

# Dự đoán nhãn cho tập test
y_pred = rf_classifier.predict(X_test_vec)

# Đánh giá hiệu suất của mô hình
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.67
Confusion Matrix:
[[ 45  95   4]
 [  9 270  17]
 [  2  50  50]]
Classification Report:
              precision    recall  f1-score   support

          -1       0.80      0.31      0.45       144
           0       0.65      0.91      0.76       296
           1       0.70      0.49      0.58       102

    accuracy                           0.67       542
   macro avg       0.72      0.57      0.60       542
weighted avg       0.70      0.67      0.64       542



In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Khởi tạo và huấn luyện mô hình Random Forest
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

# Dự đoán nhãn cho tập test
y_pred = rf_classifier.predict(X_test_tfidf)

# Đánh giá hiệu suất của mô hình
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Accuracy: 0.67
Confusion Matrix:
[[ 42 100   2]
 [ 11 275  10]
 [  1  53  48]]
Classification Report:
              precision    recall  f1-score   support

          -1       0.78      0.29      0.42       144
           0       0.64      0.93      0.76       296
           1       0.80      0.47      0.59       102

    accuracy                           0.67       542
   macro avg       0.74      0.56      0.59       542
weighted avg       0.71      0.67      0.64       542



In [17]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
rf_classifier = RandomForestClassifier(random_state=42)

# Sử dụng GridSearchCV để tìm kiếm tham số tốt nhất
grid_search = GridSearchCV(rf_classifier, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_tfidf, y_train)

# In ra tham số tốt nhất
print("Tham số tốt nhất:", grid_search.best_params_)

# Sử dụng mô hình tốt nhất để dự đoán và đánh giá trên tập test
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_tfidf)

# Đánh giá hiệu suất của mô hình
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print("Confusion Matrix:")
print(conf_matrix)
print("Classification Report:")
print(class_report)

Tham số tốt nhất: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Accuracy: 0.68
Confusion Matrix:
[[ 41 101   2]
 [  8 280   8]
 [  1  53  48]]
Classification Report:
              precision    recall  f1-score   support

          -1       0.82      0.28      0.42       144
           0       0.65      0.95      0.77       296
           1       0.83      0.47      0.60       102

    accuracy                           0.68       542
   macro avg       0.76      0.57      0.60       542
weighted avg       0.73      0.68      0.64       542



In [18]:
from sklearn.ensemble import BaggingClassifier,VotingClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression

In [19]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

In [21]:
base_models = [
    ('lr', LogisticRegression()),
    ('svm', SVC(kernel='linear', probability=True)),
    ('nb', MultinomialNB())
]
ensemble = VotingClassifier(base_models)
# Tạo BaggingClassifier với các mô hình cơ sở
bagging_model = BaggingClassifier(ensemble, n_estimators=20, random_state=42)

# Huấn luyện mô hình
bagging_model.fit(X_train_vec, y_train)


# Dự đoán và đánh giá mô hình
y_pred = bagging_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print(class_report)

Accuracy: 0.7214022140221402
              precision    recall  f1-score   support

          -1       0.72      0.51      0.60       144
           0       0.72      0.87      0.79       296
           1       0.72      0.60      0.65       102

    accuracy                           0.72       542
   macro avg       0.72      0.66      0.68       542
weighted avg       0.72      0.72      0.71       542



In [22]:
base_models = [
    ('lr', LogisticRegression()),
    ('svm', SVC(kernel='linear', probability=True)),
    ('nb', MultinomialNB())
]
ensemble = VotingClassifier(base_models)
# Tạo BaggingClassifier với các mô hình cơ sở
bagging_model = BaggingClassifier(ensemble, n_estimators=30
                                  , random_state=42)

# Huấn luyện mô hình
bagging_model.fit(X_train_vec, y_train)


# Dự đoán và đánh giá mô hình
y_pred = bagging_model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
class_report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print(class_report)

Accuracy: 0.7177121771217713
              precision    recall  f1-score   support

          -1       0.70      0.51      0.59       144
           0       0.72      0.86      0.78       296
           1       0.72      0.60      0.65       102

    accuracy                           0.72       542
   macro avg       0.71      0.66      0.68       542
weighted avg       0.72      0.72      0.71       542

